In [1]:
import numpy as np

In [23]:
A = np.array([1, 2, 3])
print(A.shape)

(3,)


In [24]:
B = np.array([4, 5, 6])
print(B.shape)

(3,)


In [29]:
D= np.array([7, 8, 9])
print(B.shape)

(3,)


In [33]:
C = np.hstack((np.expand_dims(A, axis = 1)*np.expand_dims(D, axis = 1), np.expand_dims(B, axis = 1), np.expand_dims(D, axis = 1)))

In [34]:
C

array([[ 7,  4,  7],
       [16,  5,  8],
       [27,  6,  9]])

In [26]:
C = np.array([A,B]).T

In [27]:
C

array([[1, 4],
       [2, 5],
       [3, 6]])

In [ ]:
def LucasKanade(It, It1, rect, thresh=.025, maxIt=100):
    
    '''
    Q1.1: Lucas-Kanade Forward Additive Alignment with Translation Only
    
      Inputs: 
        It: template image
        It1: Current image
        rect: Current position of the object
        (top left, bottom right coordinates, x1, y1, x2, y2)
        thresh: Stop condition when dp is too small
        maxIt: Maximum number of iterations to run
        
      Outputs:
        p: movement vector dx, dy
    '''

    # Set thresholds (you probably want to play around with the values)
    p = np.zeros(2) # dx, dy
    threshold = thresh
    maxIters = maxIt
    i = 0
    x1, y1, x2, y2 = rect
    dx = p[0]
    dy = p[1]
    
    # ----- TODO -----
    X1 = np.arange(0, It1.shape[0], 1)
    Y1 = np.arange(0, It1.shape[1], 1)   
    It1_spline = RectBivariateSpline(X1,Y1,It1)
    
    X = np.arange(0, It.shape[0], 1)
    Y = np.arange(0, It.shape[1], 1)   
    It_spline = RectBivariateSpline(X,Y,It)
    xt_v = np.arange(x1, x2 + 0.1)
    yt_v = np.arange(y1, y2 + 0.1)
    xt, yt = np.meshgrid(xt_v, yt_v)
    template = It_spline.ev(yt, xt)
    delp_m = np.finfo('float').max
    
    while (i < maxIters and delp_m > thresh):
        xt1_v = np.arange(x1 + dx, x2 + dx + 0.1)
        yt1_v = np.arange(y1 + dy, y2 + dy + 0.1)
        xt1, yt1 = np.meshgrid(xt1_v, yt1_v)
        patch_It1 = It1_spline.ev(yt1, xt1)
        
        dIt1x = It1_spline.ev(yt1, xt1, dx=0, dy=1).flatten()
        dIt1y = It1_spline.ev(yt1, xt1, dx=1, dy=0).flatten()
        A = np.hstack((np.expand_dims(dIt1x, axis = 1), np.expand_dims(dIt1y, axis = 1)))
        b = template - patch_It1
        dp = np.linalg.lstsq(A,b.flatten(),rcond=None)[0]
        dx = dx + dp[0]
        dy = dy + dp[1]
        delp_m = np.sqrt(dp[0]**2 + dp[1]**2)
        i = i + 1
    
    p[0] = dx
    p[1] = dy
    return p